In [1]:
import numpy as np

# Generalized RAS implementation

In [6]:
def gras(A, u_target, v_target, max_iter=1000, tol=1e-6):
    A = A.astype(float)
    n, m = A.shape

    # Decompose A into positive and negative parts
    P = np.maximum(A, 0)
    N = np.maximum(-A, 0)

    # Initialize multipliers
    u = np.ones(n)
    v = np.ones(m)

    for _ in range(max_iter):
        # Compute intermediate matrix
        X_pos = u[:, None] * P * v[None, :]
        X_neg = u[:, None] * N * v[None, :]
        X = X_pos - X_neg

        row_sums = X.sum(axis=1)
        col_sums = X.sum(axis=0)

        # Check convergence
        if np.allclose(row_sums, u_target, atol=tol) and np.allclose(col_sums, v_target, atol=tol):
            break

        # Update row multipliers u
        denom_row = (P @ v - N @ v)
        u = np.divide(u_target, denom_row, out=np.ones_like(u), where=denom_row != 0)

        # Update column multipliers v
        denom_col = (P.T @ u - N.T @ u)
        v = np.divide(v_target, denom_col, out=np.ones_like(v), where=denom_col != 0)

    else:
        print("Warning: GRAS did not converge within the maximum number of iterations.")

    # Final matrix
    X = (u[:, None] * P * v[None, :]) - (u[:, None] * N * v[None, :])
    return X

# MR-GRAS implementation
from https://pure.rug.nl/ws/files/178379286/17421772.2020.pdf

In [23]:
# --- Helper functions ---
def inv_diag(v):
    """Inverse of a diagonal vector as a matrix."""
    inv = np.divide(1.0,
                    v,
                    out=np.ones_like(v),
                    where=(v != 0))
    return np.diag(inv)

def inv_elementwise(X):
    """Element-wise inverse with safeguard."""
    inv = np.divide(1.0,
                    X,
                    out=np.ones_like(X),
                    where=(X != 0))
    return inv

# --- MR-GRAS implementation ---
def mr_gras(X0, u, v, G, Q, W, eps=1e-6, max_iter=1000):

    """
    X0: original matrix to balance
    u: target of row totals
    v: target of column totals
    
    """

    X0 = np.asarray(X0, dtype=float)
    m, n = X0.shape
    h, k = W.shape

    # Decompose X0 into positive (P) and negative (N) parts
    N = np.where(X0 < 0, -X0, 0)
    P = X0 + N

    # Initial multipliers
    r = np.ones(m)
    s = np.ones(n)
    T = np.ones((h, k))

    for iteration in range(max_iter):
        # Expand T to matrix shape: Te = G' * T * Q'
        Te = G.T @ T @ Q.T

        # Compute column multipliers s
        p_rt = (P * Te).T @ r
        n_rt = (N * inv_elementwise(Te)).T @ inv_diag(r) @ np.ones(m)
        s_new = np.divide(v + np.sqrt(v**2 + 4 * p_rt * n_rt), 2 * p_rt, out=-n_rt / v, where=(p_rt != 0))

        # Compute row multipliers r
        p_st = (P * Te) @ s_new
        n_st = (N * inv_elementwise(Te)) @ inv_diag(s_new) @ np.ones(n)
        r_new = np.divide(u + np.sqrt(u**2 + 4 * p_st * n_st), 2 * p_st, out=-n_st / u, where=(p_st != 0))

        # Update aggregation multipliers T
        P_rs = G @ (np.diag(r_new) @ P @ np.diag(s_new)) @ Q
        N_rs = G @ (inv_diag(r_new) @ N @ inv_diag(s_new)) @ Q

        T_new = np.divide(W + np.sqrt(W**2 + 4 * P_rs * N_rs), 2 * P_rs, out=-N_rs / W, where=(P_rs != 0))
        T_new[W == 1010101] = 1  # Handle missing W constraints

        # Check convergence
        max_change = max(
            np.max(np.abs(s_new - s)),
            np.max(np.abs(r_new - r)),
            np.max(np.abs(T_new - T))
        )
        if max_change < eps:
            break

        # Update variables
        s, r, T = s_new, r_new, T_new

    # Final matrix reconstruction
    Te = G.T @ T @ Q.T
    X = Te * (np.diag(r) @ P @ np.diag(s)) - inv_elementwise(Te) * (inv_diag(r) @ N @ inv_diag(s))
    return X, r, s, T

In [ ]:
# Example from the supplementary information
X0 = np.array(
    [
        [63, 9, 14, 9, -18, 75],
        [-14, 53, -10, 66, 69, 66],
        [16, 56, -21, 9, 93, -25],
        [53, 16, 74, 72, -1, 80],
        [4, -48, 14, 64, 51, 99],
        [61, -1, 84, 6, 16, 27]
    ]
)

In [45]:
G = np.kron([1, 1], np.eye(3))
Q = G.T
print(X0, Q, G)

[[ 63   9  14   9 -18  75]
 [-14  53 -10  66  69  66]
 [ 16  56 -21   9  93 -25]
 [ 53  16  74  72  -1  80]
 [  4 -48  14  64  51  99]
 [ 61  -1  84   6  16  27]] [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] [[1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 1.]]


In [ ]:
# Initial totals for products/sectors
# this is the aggregated, single-region format of the MRIO table
W0 = G@X0@Q
W0

array([[197.,   6., 243.],
       [120., 125., 169.],
       [ 92., 164.,  65.]])

In [50]:
# Targets
u = np.array(
    [160, 194, 145, 320, 134, 151]
)
v = np.array(
    [197, 71, 151, 242, 178, 265]
)
W = np.array([
    [230, 0, 250],
    [123, 75, 130],
    [86, 174, 36]
])

In [51]:
X, r, s, t = mr_gras(X0, u, v, G, Q, W)

C:\Users\Gibon\AppData\Local\Temp\ipykernel_15248\765451770.py:59: RuntimeWarning: divide by zero encountered in divide
  T_new = np.divide(W + np.sqrt(W**2 + 4 * P_rs * N_rs), 2 * P_rs, out=-N_rs / W, where=(P_rs != 0))


In [52]:
X

array([[ 74.24551328,   8.24153086,  16.36994349,  10.55612892,
        -21.51321699,  72.10009714],
       [-13.42201347,  44.35933243, -10.39910992,  68.51518149,
         52.76670684,  52.17989595],
       [ 18.7776562 ,  64.75060578, -19.31882811,  10.51226793,
         98.2518022 , -27.97345121],
       [ 61.73297864,  14.48095022,  85.5189899 ,  83.46537917,
         -1.20926408,  76.01096947],
       [  4.01245475, -59.63377668,  12.94707482,  63.89437723,
         37.5077374 ,  75.27213915],
       [ 51.65353019,  -1.19865836,  65.88196969,   5.05654568,
         12.19625039,  17.41030963]])

In [53]:
X.sum(1)

array([159.99999669, 193.99999333, 145.00005279, 320.00000331,
       134.00000667, 150.99994721])